In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/Users/Jenny/Desktop/seoulbigcam/Raw Data/서울시 지역아동센터 시설현황정보.csv', encoding = 'CP949')
df.head(2)

,시설ID,시설명,서비스분류코드,서비스분류,자치구코드,자치구명,연령구분코드,연령구분,X좌표값,Y좌표값,...,사용료,학기시작시간,학기종료시간,방학시작시간,방학종료시간,토요일운영여부,토요일운영시작시간,토요일운영종료시간,등록일시,수정일시
0,DM230901,희망주는 지역아동센터,1003,지역아동센터,11230,동대문구,1003,초등학생,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,2023-09-05 16:55:00.0,NaN
1,GB230802,아름다운땅지역아동센터,1003,지역아동센터,11305,강북구,1003,초등학생,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,2023-08-01 16:55:01.0,NaN


In [4]:
center = df[['시설명', '자치구명', 'X좌표값', 'Y좌표값', '기본주소']]

##### 결측치 정보 수정

In [5]:
center_na = center[center['기본주소'].isna()]
center_na

,시설명,자치구명,X좌표값,Y좌표값,기본주소
0,희망주는 지역아동센터,동대문구,NaN,NaN,NaN
1,아름다운땅지역아동센터,강북구,NaN,NaN,NaN
2,동산비젼스쿨지역아동센터,강북구,NaN,NaN,NaN
3,구로구공립지역아동센터,구로구,NaN,NaN,NaN
4,방화지역아동센터,강서구,NaN,NaN,NaN
5,열린지역아동센터,마포구,NaN,NaN,NaN
6,좋은친구들지역아동센터,동작구,NaN,NaN,NaN
7,좋은친구들지역아동센터,동작구,NaN,NaN,NaN
8,좋은친구들지역아동센터,동작구,NaN,NaN,NaN


In [6]:
center_na = center_na.drop([2, 6, 7, 8])
center_na = center_na.reset_index()
center_na

,index,시설명,자치구명,X좌표값,Y좌표값,기본주소
0,0,희망주는 지역아동센터,동대문구,NaN,NaN,NaN
1,1,아름다운땅지역아동센터,강북구,NaN,NaN,NaN
2,3,구로구공립지역아동센터,구로구,NaN,NaN,NaN
3,4,방화지역아동센터,강서구,NaN,NaN,NaN
4,5,열린지역아동센터,마포구,NaN,NaN,NaN


In [7]:
names = ['희망주는 지역아동센터', '아름다운땅지역아동센터', '구로구공립지역아동센터', '방화지역아동센터', '열린지역아동센터']
addrs = ['서울특별시 동대문구 한천로 85','서울특별시 강북구 오현로 194', '서울특별시 구로구 천왕로 32', '서울특별시 강서구 양천로6길 28', '서울특별시 마포구 월드컵북로48길 8']

center_na2 = pd.DataFrame({'시설명':names, '기본주소':addrs})
center_na2

,시설명,기본주소
0,희망주는 지역아동센터,서울특별시 동대문구 한천로 85
1,아름다운땅지역아동센터,서울특별시 강북구 오현로 194
2,구로구공립지역아동센터,서울특별시 구로구 천왕로 32
3,방화지역아동센터,서울특별시 강서구 양천로6길 28
4,열린지역아동센터,서울특별시 마포구 월드컵북로48길 8


In [8]:
# 주소로 위도, 경도 좌표 추출하기

addrs = center_na2['기본주소']

# 가입 없이 주소->좌표 변환
from geopy.geocoders import Nominatim

list = []

def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    crd = {"lat": str(geo.latitude), "lng": str(geo.longitude)}

    return crd

for i in range(5):
    crd = geocoding(addrs[i])
    list.append(crd)

In [9]:
center_na3 = pd.DataFrame(list)
center_na3

,lat,lng
0,37.60589,127.06661
1,37.6295518,127.0394819
2,37.4803508,126.8397388
3,37.5707907,126.8074712
4,37.5772493,126.8952066


In [10]:
center_na2['Y좌표값'] = center_na3['lat']
center_na2['X좌표값'] = center_na3['lng']
center_na2

,시설명,기본주소,Y좌표값,X좌표값
0,희망주는 지역아동센터,서울특별시 동대문구 한천로 85,37.60589,127.06661
1,아름다운땅지역아동센터,서울특별시 강북구 오현로 194,37.6295518,127.0394819
2,구로구공립지역아동센터,서울특별시 구로구 천왕로 32,37.4803508,126.8397388
3,방화지역아동센터,서울특별시 강서구 양천로6길 28,37.5707907,126.8074712
4,열린지역아동센터,서울특별시 마포구 월드컵북로48길 8,37.5772493,126.8952066


In [11]:
center = center.dropna()
center = pd.concat([center, center_na2], ignore_index=True)
center

,시설명,자치구명,X좌표값,Y좌표값,기본주소
0,꿈나무들의둥지지역아동센터,양천구,126.86637,37.54423,서울특별시 양천구 목동중앙북로8길 104
1,송천동지역아동센터,강북구,127.02509,37.61515,서울특별시 강북구 삼양로20길 53
2,들꽃청소년지역아동센터,강동구,127.12642,37.54553,서울특별시 강동구 천중로 39
3,꿈꾸는땅지역아동센터,강동구,127.1767,37.57272,서울특별시 강동구 아리수로93나길 90
4,천호지역아동센터,강동구,127.13211,37.54341,서울특별시 강동구 구천면로 260
...,...,...,...,...,...
418,희망주는 지역아동센터,NaN,127.06661,37.60589,서울특별시 동대문구 한천로 85
419,아름다운땅지역아동센터,NaN,127.0394819,37.6295518,서울특별시 강북구 오현로 194
420,구로구공립지역아동센터,NaN,126.8397388,37.4803508,서울특별시 구로구 천왕로 32
421,방화지역아동센터,NaN,126.8074712,37.5707907,서울특별시 강서구 양천로6길 28


In [12]:
import pandas as pd
import requests

locations = []

def get_location(row):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={}&y={}'.format(row['X좌표값'], row['Y좌표값'])
    headers = {
        "Authorization": "KakaoAK 8cd64c4299a983cac6fd3edef103f134"
    }
    response = requests.get(url, headers=headers)
    
    try:
        response_json = response.json()
        place = response_json['documents'][1]
        city = place.get('region_1depth_name')
        gu = place.get('region_2depth_name')
        dong = place.get('region_3depth_name')
        locations.append({'city': city, 'gu': gu, 'dong': dong})
    except (IndexError, AttributeError, KeyError):
        print('API 호출 오류:', response.text)


# 데이터프레임의 각 행에 대해 get_location 함수 호출
for i, row in center.iterrows():
    get_location(row)
    
    # 50개씩 처리될 때 메시지 출력
    if i % 200 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

# 결과를 데이터프레임에 추가
center['city'] = [location.get('city') for location in locations]
center['gu'] = [location.get('gu') for location in locations]
center['dong'] = [location.get('dong') for location in locations]

0 번째 경위도 좌표계에 대한 정보 불러오는 중...
200 번째 경위도 좌표계에 대한 정보 불러오는 중...
400 번째 경위도 좌표계에 대한 정보 불러오는 중...


In [13]:
center

,시설명,자치구명,X좌표값,Y좌표값,기본주소,city,gu,dong
0,꿈나무들의둥지지역아동센터,양천구,126.86637,37.54423,서울특별시 양천구 목동중앙북로8길 104,서울특별시,양천구,목2동
1,송천동지역아동센터,강북구,127.02509,37.61515,서울특별시 강북구 삼양로20길 53,서울특별시,강북구,송천동
2,들꽃청소년지역아동센터,강동구,127.12642,37.54553,서울특별시 강동구 천중로 39,서울특별시,강동구,천호2동
3,꿈꾸는땅지역아동센터,강동구,127.1767,37.57272,서울특별시 강동구 아리수로93나길 90,서울특별시,강동구,강일동
4,천호지역아동센터,강동구,127.13211,37.54341,서울특별시 강동구 구천면로 260,서울특별시,강동구,천호1동
...,...,...,...,...,...,...,...,...
418,희망주는 지역아동센터,NaN,127.06661,37.60589,서울특별시 동대문구 한천로 85,서울특별시,동대문구,이문2동
419,아름다운땅지역아동센터,NaN,127.0394819,37.6295518,서울특별시 강북구 오현로 194,서울특별시,강북구,번2동
420,구로구공립지역아동센터,NaN,126.8397388,37.4803508,서울특별시 구로구 천왕로 32,서울특별시,구로구,오류2동
421,방화지역아동센터,NaN,126.8074712,37.5707907,서울특별시 강서구 양천로6길 28,서울특별시,강서구,방화2동


In [14]:
center.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423 entries, 0 to 422
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시설명     423 non-null    object
 1   자치구명    418 non-null    object
 2   X좌표값    423 non-null    object
 3   Y좌표값    423 non-null    object
 4   기본주소    423 non-null    object
 5   city    423 non-null    object
 6   gu      423 non-null    object
 7   dong    423 non-null    object
dtypes: object(8)
memory usage: 26.6+ KB


In [15]:
center_new = center.groupby(['gu', 'dong'], as_index=False)['시설명'].count()
center_new.sort_values('시설명')

,gu,dong,시설명
0,강남구,세곡동,1
133,마포구,도화동,1
136,마포구,서교동,1
137,마포구,성산2동,1
138,마포구,신수동,1
...,...,...,...
233,종로구,창신2동,4
82,노원구,상계1동,4
38,관악구,미성동,4
74,금천구,독산2동,4


In [17]:
# 행정동별 면적 데이터 로드

area = pd.read_csv("/Users/Jenny/Desktop/seoulbigcam/Raw Data/행정동별 면적.csv")


area = area.drop(area.index[0:3])     # 필요없는 데이터 정리
area = area.reset_index(drop=True)    # 인덱스 리셋
area = area.drop('동별(1)', axis=1)    # 필요없는 컬럼 삭제
area = area.rename(columns={'동별(2)': 'gu',
                            '동별(3)':'dong',
                            '2022':'area_km2'})     # 컬럼명 변경

area.head()

,gu,dong,area_km2
0,종로구,소계,23.91
1,종로구,사직동,1.23
2,종로구,삼청동,1.49
3,종로구,부암동,2.27
4,종로구,평창동,8.87


In [18]:
center_merge = center_new.merge(area[['gu', 'dong', 'area_km2']], on=['gu','dong'], how = 'left')
center_merge['area_km2'] = center_merge['area_km2'].astype(float)
center_merge['density_supcent'] = center_merge['시설명'] / center_merge['area_km2']
center_merge

,gu,dong,시설명,area_km2,density_supcent
0,강남구,세곡동,1,6.36,0.157233
1,강남구,일원1동,1,0.92,1.086957
2,강동구,강일동,2,2.26,0.884956
3,강동구,고덕1동,1,1.51,0.662252
4,강동구,고덕2동,2,2.50,0.800000
...,...,...,...,...,...
246,중랑구,상봉2동,1,0.65,1.538462
247,중랑구,신내1동,1,2.56,0.390625
248,중랑구,신내2동,1,0.93,1.075269
249,중랑구,중화1동,1,0.64,1.562500


In [19]:
center_merge.sort_values('density_supcent')

,gu,dong,시설명,area_km2,density_supcent
228,은평구,진관동,1,11.53,0.086730
37,관악구,대학동,1,8.30,0.120482
149,서초구,양재2동,1,7.58,0.131926
0,강남구,세곡동,1,6.36,0.157233
22,강북구,우이동,2,10.28,0.194553
...,...,...,...,...,...
20,강북구,수유2동,3,0.57,5.263158
229,종로구,교남동,2,0.35,5.714286
67,구로구,구로4동,3,0.46,6.521739
74,금천구,독산2동,4,0.60,6.666667


In [20]:
center_f = center_merge[['gu', 'dong', 'density_supcent']]
center_f.to_csv('/Users/Jenny/Desktop/seoulbigcam/Final Data/지역아동센터밀집도.csv')

In [21]:
center_merge.to_csv('/Users/Jenny/Desktop/seoulbigcam/Preprocessed Data/서울시 행정동별 지역아동센터 밀집도.csv')

In [ ]:
center_f = center_merge[['gu', 'dong', 'density_supcent']]

In [18]:
center_merge = pd.read_csv('/Users/Jenny/Desktop/seoulbigcam/Preprocessed Data/서울시 행정동별 지역아동센터 밀집도.csv')

In [19]:
center_merge = center_merge.rename(columns={'gu': '구', 'dong': '동',
                             '시설명': '센터 수',
                             'area_km2':'면적(km2)',
                             'density_supcent':'면적당 아동센터 수'})

In [20]:
center_f = center_merge[['구', '동', '센터 수', '면적(km2)', '면적당 아동센터 수']]
center_f.to_csv('Final Data/교육_지역아동센터_밀집도_동별_2023.csv',index=False)